# MBTI별 포스팅 모으기 

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request

In [2]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

#input_file = r'.\data\training'
#allFile_list = glob.glob(os.path.join(input_file, 'mbti_*'))
#for file in allFile_list:
#    csv = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
#    cleanMbti = csv['posts'].str.replace('[^A-Za-z\s]+', '')

# OverSampling

In [3]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

from imblearn.over_sampling import RandomOverSampler

In [4]:
mbti=pd.read_csv('./data/mbti_1.csv')

types = mbti.groupby('type').count()

output_file = '.\data\\training\\mbti.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
allData = []

for type in types.index:
    condition = mbti['type'] == type # condition: mbti['type']가 topfive의 원소인 type 같을 bool 조건
    ownsentence = mbti[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    allData.append(ownsentence)
dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

print(types)

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
y = y_data.values.reshape(-1, 1)

X_resampled, y_resampled = RandomOverSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


      posts
type       
ENFJ    190
ENFP    675
ENTJ    231
ENTP    685
ESFJ     42
ESFP     48
ESTJ     39
ESTP     89
INFJ   1470
INFP   1832
INTJ   1091
INTP   1304
ISFJ    166
ISFP    271
ISTJ    205
ISTP    337


Counter({'INFJ': 1832,
         'ENTP': 1832,
         'INTP': 1832,
         'INTJ': 1832,
         'ENTJ': 1832,
         'ENFJ': 1832,
         'INFP': 1832,
         'ENFP': 1832,
         'ISFP': 1832,
         'ISTP': 1832,
         'ISFJ': 1832,
         'ISTJ': 1832,
         'ESTP': 1832,
         'ESFP': 1832,
         'ESTJ': 1832,
         'ESFJ': 1832})

# tokenize and stemming

In [5]:
mbti= dataCombine
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [6]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료


0

In [8]:
#vectorization (tfidf)
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData).toarray().astype(np.uint8)
y = np.array(y_data)


[[0.16126    0.         0.         ... 0.         0.         0.        ]
 [0.15308429 0.         0.         ... 0.         0.         0.        ]
 [0.0701916  0.         0.         ... 0.         0.         0.        ]
 ...
 [0.14387306 0.         0.         ... 0.         0.         0.        ]
 [0.25156428 0.         0.         ... 0.         0.         0.        ]
 [0.20342093 0.         0.         ... 0.         0.         0.        ]]


# XGBoost_Oversampling

In [9]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [10]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train)

Model building Start


MemoryError: Unable to allocate 19.4 GiB for an array with shape (8794, 295647) and data type float64

In [ ]:
preds = model.predict(X_test)
print("Model building End")

In [ ]:
import joblib

joblib.dump(model, 'XGBoost_oversample.pkl')

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/